In [2]:
# Import libraries
import pystan # install with pip install pystan
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import beta
import matplotlib.pyplot as plt
import pickle
import hashlib
import time

def get_stan_model(code):
    code_hash = hashlib.sha1(code.encode('utf-8')).hexdigest()
    cache_path =  './cache/' + code_hash + '.pkl'
    try:
        with open(cache_path, 'rb') as file:
            model = pickle.load(file)
    except Exception:
        model = pystan.StanModel(model_code=code) ## c로 변환
        with open(cache_path, 'wb') as file:
            pickle.dump(model, file)
    return model

## 8.1 계층모델 도입

네 회사 사원 41명의 나이(X)와 연봉(Y)

In [12]:
#data
df = pd.read_csv('data/data-salary-2.csv')

In [13]:
df.groupby('KID').describe()

X                                                          Y  \
    count       mean       std   min    25%   50%    75%   max count   
KID                                                                    
1    15.0  13.933333  8.455486   1.0   6.00  16.0  20.00  26.0  15.0   
2    12.0  11.916667  6.666856   2.0   6.75  11.0  17.25  22.0  12.0   
3    10.0  19.600000  5.399588  11.0  15.75  19.0  24.25  27.0  10.0   
4     3.0  24.666667  3.055050  22.0  23.00  24.0  26.00  28.0   3.0   

                                                                  
           mean         std    min     25%    50%     75%    max  
KID                                                               
1    491.800000   68.421175  380.0  453.00  484.0  526.50  610.0  
2    565.333333  133.610923  376.0  442.25  545.5  666.75  763.0  
3    558.100000   74.616724  411.0  523.75  541.0  615.25  666.0  
4    725.333333    3.055050  722.0  724.00  726.0  727.00  728.0

네 회사 모두 연공서열형 임금제도지만, 그 증가비율이 다르다.

네 번째 회사는 데이터가 3개뿐이므로 추정이 어려우며 단순 회귀시 음수 기울기(-1)를 가져 기본 가정에 모순

이는 overfitting의 결과로 설명가능

1. 그룹차의 고려
2. 데이터가 부족한 특정 그룹의 존재 

가 계층모형의 도입배경이다.

In [14]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
df4 = df[df.KID == 4]
intercept = np.ones(df4.shape[0])
model.fit(np.c_[intercept, df4.X], df4.Y)
model.coef_

array([ 0., -1.])

## 그룹차 고려하지 않는 complete pooling

$$Y[n] = Normal(a + bX[n], \sigma_Y)$$

In [15]:
#data
df = pd.read_csv('data/data-salary-2.csv')

#model
model_8_1 ='''
data {
    int N;
    vector [N] X;
    vector [N] Y;
}

parameters {
    real a;
    real b;
    real <lower = 0> sigma;
}

model {
    Y ~ normal(a + b * X, sigma);
}
        

'''

#python interface

data = {
    'N': df.shape[0],
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_1 = get_stan_model(model_8_1)
params = model_8_1.sampling(data = data)


sigma값이 크게 추정 -> 회사(그룹)차가 고려된 모형 필요!

## 그룹별로 절편과 기울기를 가지는 no pooling

$$Y[n] = Normal(a[KID[n]] + b[KID[n]] * X[n], \sigma_Y)$$

In [16]:
#data
df = pd.read_csv('data/data-salary-2.csv')

#model
model_8_2 ='''
data {
    int N;
    int K;
    int <lower =1, upper = K> KID[N];
    vector [N] X;
    vector [N] Y;
}

parameters {
    real a[K];
    real b[K];
    real <lower = 0> sigma;
}

model {
    for (n in 1:N)
        Y[n] ~ normal(a[KID[n]] + b[KID[n]] * X[n], sigma);
        
}
'''

#python interface

data = {
    'N': df.shape[0],
    'K': len(np.unique(df.KID)),
    'KID': df.KID,
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_2 = get_stan_model(model_8_2)
params = model_8_2.sampling(data = data)

# stan에서 인덱스는 무조건 array로 (NO VECTOR) 받게 되어 있어 for문 한번은 돌아야하며, 이때 X,Y는 real array, vector모두 가능

다음 두 가지 경우의 추정이 어려워
1. 새로운 회사의 절편과 기울기에 대한 추정
2. 데이터가 적은 경우의 추정 (회사4)

partial pooling인 계층모형을 이용하게 된다.


$$Y[n] = Normal(a[KID[n]] + b[KID[n]] * X[n], \sigma_Y)$$

$$a[k] = a_{전체평균} + a_{회사차}[k]$$

$$a_{회사차} \sim  Normal(0, \sigma_a)$$

$$b[k] = b_{전체평균} + b_{회사차}[k]$$

$$b_{회사차}  \sim Normal(0, \sigma_b)$$

In [17]:
#data
df = pd.read_csv('data/data-salary-2.csv')

#model
model_8_3 ='''
data {
    int N;
    int K;
    int <lower =1, upper = K> KID[N];
    real X[N];
    real Y[N];
}

parameters {
    real a0;
    real b0;
    real ak[K];
    real bk[K];
    real<lower=0> s_a;
    real<lower=0> s_b;
    real<lower=0> s_Y;
}

transformed parameters{
    real a[K];
    real b[K];
    for (k in 1:K) {
        a[k] = a0 + ak[k];
        b[k] = b0 + bk[k];
    }
}
model {
    for (k in 1:K){
        ak[k] ~ normal(0, s_a);
        bk[k] ~ normal(0, s_b);
    }
    for (n in 1:N)
        Y[n] ~ normal(a[KID[n]] + b[KID[n]]*X[n], s_Y);    
}
'''

#python interface

data = {
    'N': df.shape[0],
    'K': len(np.unique(df.KID)),
    'KID': df.KID,
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_3 = get_stan_model(model_8_3)
params = model_8_3.sampling(data = data)

# transformed parameters 블록에는 변수치환이므로 ~ 가 아닌 =를 사용해야함에 주의!

모델 등가표현에 따라 다음과 같은 표현도 가능하며, stan에서는 모델 표현에 따라 계산속도와 수렴여부차이가 크다.

model_8_3에서는 회사차를 평균 0의 정규분포에서 추출한 후 이를 transformed parameters block에서 회사별 회귀계수로 변환했지만, 

model_8_3에서는 model block에서 바로 각 회사별 회귀계수를 추출한다.

$$Y[n] = Normal(a[KID[n]] + b[KID[n]] * X[n], \sigma_Y)$$

$$a[k] \sim  Normal(a_{전체평균}, \sigma_a)$$


$$b[k]  \sim Normal(b_{전체평균}, \sigma_b)$$

In [16]:
#data
df = pd.read_csv('data/data-salary-2.csv')

#model
model_8_4 ='''
data {
    int N;
    int K;
    real X[N];
    real Y[N];
    int <lower =1, upper = K> KID[N];
}

parameters {
    real a0;
    real b0;
    real a[K];
    real b[K];
    real<lower=0> s_a;
    real<lower=0> s_b;
    real<lower=0> s_Y;
}

model {
    for (k in 1:K){
        a[k] ~ normal(a0, s_a);
        b[k] ~ normal(b0, s_b);
    }
    for (n in 1:N)
        Y[n] ~ normal(a[KID[n]] + b[KID[n]]*X[n], s_Y);    
}
'''

#python interface

data = {
    'N': df.shape[0],
    'K': len(np.unique(df.KID)),
    'KID': df.KID,
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_4 = get_stan_model(model_8_4)
params = model_8_4.sampling(data = data)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d89324136efea1f09c1c6a0eb1a3fa12 NOW.


## 8.2 복수의 계층이 있는 계층모델

회사 ID인 KID에 더해 회사가 소속된 4개 업계의 ID, GID가 추가되었다.

$$Y[n] = Normal(a[KID[n]] + b[KID[n]] * X[n], \sigma_Y)$$

$$a[k] \sim  Normal(a_{업계평균}[K2G[k]], \sigma_{ag})$$

$$b[k]  \sim Normal(b_{업계평균}[K2G[k]], \sigma_{bg})$$

$$a_{업계평균}[g] \sim  Normal(a_{전체평균}, \sigma_a)$$

$$b_{업계평균}[g] \sim Normal(b_{전체평균}, \sigma_b)$$

In [20]:
#data
df = pd.read_csv('data/data-salary-3.txt')

#model
model_8_5 ='''
data {
    int N;
    int K;
    int G;
    real X[N];
    real Y[N];
    int <lower =1, upper = K> KID[N];
    int <lower =1, upper = G> K2G[K];
}

parameters {
    real a0;
    real b0;
    real a[G];
    real b[G];
    real ak[K];
    real bk[K];
    real<lower=0> s_ag;
    real<lower=0> s_bg;
    real<lower=0> s_a;
    real<lower=0> s_b;
    real<lower=0> s_Y;
}

model {
    for (g in 1:G){
        a[g] ~ normal(a0, s_ag);
        b[g] ~ normal(b0, s_bg);
    }

    for (k in 1:K){
        ak[k] ~ normal(a[K2G[k]], s_a);
        bk[k] ~ normal(b[K2G[k]], s_b);
    }
    for (n in 1:N)
        Y[n] ~ normal(ak[KID[n]] + bk[KID[n]]*X[n], s_Y);    
}
'''
# K = df.KID
# G = df.GID
# K2G = dict(zip(K, G))
# K2G = [j for _, j in K2G.items()]
K2G = df[['KID', 'GID']].drop_duplicates()['GID']
#python interface

data = {
    'N': df.shape[0],
    'K': len(np.unique(df.KID)),
    'G': len(np.unique(df.GID)),
    'KID': df.KID,
    'K2G': K2G,
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_5 = get_stan_model(model_8_5)
params = model_8_5.sampling(data = data)

만약 업계별로 업계 내 회사별 분산이 다르다면(특정 산업의 독점도 등의 요인에 의해) $\sigma_{ag}에도 차별적 분산을 줄 수 있다.

연봉이 나이에 영향받는 정도가 업계별로 다르다면, 나이 이외의 요인들이 연봉에 주는 영향도 업계별로 다를 수 있다. 

나이 이외의 요인들이 연봉에 주는 영향은 노이즈에 반영되어 있고 따라서 $\sigma_Y$도 $\sigma_Y[K2G[k]]$로 바꾼다.

이를 위해 stan 모델에서는 GID[n]라는 변수가 추가되어야 한다.

$$Y[n] = Normal(a[KID[n]] + b[KID[n]] * X[n], \sigma_Y[K2G[k]])$$

$$a[k] \sim  Normal(a_{업계평균}[K2G[k]], \sigma_{a}[K2G[k]])$$

$$b[k]  \sim Normal(b_{업계평균}[K2G[k]], \sigma_{b}[K2G[k]])$$

$$a_{업계평균}[g] \sim  Normal(a_{전체평균}, \sigma_a)$$

$$b_{업계평균}[g] \sim Normal(b_{전체평균}, \sigma_b)$$



In [71]:
#data
df = pd.read_csv('data/data-salary-3.txt')

#model
model_8_7 = '''
data {
    int N;
    int K;
    int G;
    real X[N];
    real Y[N];
    int <lower =1, upper = K> KID[N];
    int <lower =1, upper = G> GID[N];
    int <lower =1, upper = G> K2G[K];
}

parameters {
    real a0;
    real b0;
    real a[G];
    real b[G];
    real ak[K];
    real bk[K];
    real<lower=0> s_ag;
    real<lower=0> s_bg;
    real<lower=0> s_a[G];
    real<lower=0> s_b[G];
    real<lower=0> s_Y[G];
}

model {
    for (g in 1:G){
        a[g] ~ normal(a0, s_ag);
        b[g] ~ normal(b0, s_bg);
    }

    for (k in 1:K){
        ak[k] ~ normal(a[K2G[k]], s_a[K2G[k]]);
        bk[k] ~ normal(b[K2G[k]], s_b[K2G[k]]);
        
    }
    for (n in 1:N)
        Y[n] ~ normal(ak[KID[n]] + bk[KID[n]]*X[n], s_Y[]);    
}
'''
K = df.KID
G = df.GID
K2G = dict(zip(K, G))
K2G = [j for _, j in K2G.items()]

#python interface

data = {
    'N': df.shape[0],
    'K': len(np.unique(df.KID)),
    'G': len(np.unique(df.GID)),
    'KID': df.KID,
    'GID': df.GID,
    'K2G': K2G,
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_8_7 = get_stan_model(model_8_7)
params = model_8_7.sampling(data = data)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a6cc1129160ffaa7951ec4d318ce95a9 NOW.


## 8.3 비선형 모델의 계층모델

약을 투여 후 경과시간(X)에 대한 혈중 농도(Y)는 비선형이며, 다음과 같이 모델링할 수 있다




$$Y[n, t] \sim  Normal(a[n]((1-exp(-b[n]Time[t])), \sigma_Y)$$

$$log(a[n]) \sim  Normal(a_{전체평균}, \sigma_{a})$$

$$log(b[n]) \sim  Normal(b_{전체평균}, \sigma_{b})$$

stan에서 log형태의 변수(log(a), log(b))는 model block에서 추출한 후 transformed parameters에서 변형(a,b)을 한 후 

y과의 관계에 반영된다. 여기서 a는 y가 점근하는 값이고, b는 해당 점근값에 얼마나 빨리 가까워지는 지를 나타낸다.

또한 환자별, 시간별로 y가 결정되므로 2중 for문이 필요하다. 

이때 stan은 C와 같이 column major이므로(동일 column 원소가 동일 row와 비교해 메모리 상에서 가까움) 시간에 대한 for문을 먼저 쓴 후 환자에 대한 for문을 썼다.


In [78]:
df = pd.read_csv('data/data-conc-2.txt')

In [76]:
#data
df = pd.read_csv('data/data-conc-2.txt')

#model
model_8_7 ='''
data {
  int N;
  int T;
  real Time[T];
  real Y[N,T];
}

parameters {
  real a0;
  real b0;
  real log_a[N];
  real log_b[N];
  real<lower=0> s_a;
  real<lower=0> s_b;
  real<lower=0> s_Y;
}

transformed parameters {
  real a[N];
  real b[N];
  for (n in 1:N) {
    a[n] = exp(log_a[n]);
    b[n] = exp(log_b[n]);
  }
}

model {
  for (n in 1:N) {
    log_a[n] ~ normal(a0, s_a);
    log_b[n] ~ normal(b0, s_b);
  }
  for (t in 1:T)
      for (n in 1:N)
          Y[n,t] ~ normal(a[n]*(1 - exp(-b[n]*Time[t])), s_Y);
}
'''

#python interface
Time =  [int(i[4:]) for i in df.columns if i[:4] == 'Time']
Y = df.set_index('PersonID').values


data = {
    'N': df.shape[0],
    'T': df.shape[1] -1,
    'Time': Time,
    'Y' : Y,
}

#result
model_8_7 = get_stan_model(model_8_7)
params = model_8_7.sampling(data = data)

In [77]:
params

Inference for Stan model: anon_model_65b76dc064ef56a8f9b1f53a56047226.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a0          2.86  1.6e-3   0.11   2.64   2.79   2.86   2.93   3.08   4745    1.0
b0         -1.18  2.0e-3   0.12  -1.42  -1.25  -1.17   -1.1  -0.95   3541    1.0
log_a[1]    2.56  1.4e-3   0.09   2.37    2.5   2.56   2.63   2.75   4683    1.0
log_a[2]    2.11  2.3e-3   0.14   1.83   2.01   2.11    2.2    2.4   3761    1.0
log_a[3]    3.17  7.2e-4   0.05   3.07   3.14   3.17    3.2   3.26   4803    1.0
log_a[4]    3.26  6.1e-4   0.05   3.16   3.23   3.25   3.29   3.34   5583    1.0
log_a[5]    2.64  1.9e-3   0.11   2.43   2.56   2.64   2.71   2.86   3531    1.0
log_a[6]    2.75  1.6e-3    0.1   2.56   2.68   2.75   2.81   2.95   3864    1.0
log_a[7]    2.69  1.3e-3   0.09   2.53   2.64   2.69   2.75   2.87   4575    1.0
l

## 8.4 로지스틱 회귀의 계층모델

5단원의 학생출결 데이터로 학생과 과목 차를 고려한 계층모형을 위해 CourseID를 추가


앞선 로지스틱회귀에서는 출석률을 학생 아르바이트 여부, 흥미도, 날씨의 회귀로 바로 표현했으나, 이는 학생별, 과목별 차이를 충분히 고려하지 못한다.
$$q[i] \sim inv\_logit(b1 + b2 * A[i] + b3 * Score[i] + b4 * Weather$$
$$Y[i] \sim Bernoulli(q[i])$$

출석확률에 영향주는 여러 요소를 크게 학생, 과목, 회별 수업에 의존하는 요소로 나눈다.

출석확률 <- 학생, 과목, 회별 수업

학생 <- A, Score, 학생 차

과목 <- 과목 차

회별 수업 <- Weather


\begin{aligned} x[i] &=b_{1}+x_{\text {학생 }}[\text { PersonID }[i]]+x_{\text{과목}}[\text { CourseID }[i]]+x_{ \text { 수업 }}[i] \\ q[i] &=inv\_logit(x[i]) \\ Y[i] & \sim \text { Bernoulli }(q[i]) \\ x_{\text {학생 }}[n] &=b_{2} A[n]+b_{3} S c o r e[n]+b_{\text{학생 차}}[n] \\ b_{\text{학생 차}}[n] & \sim \operatorname{Normal}\left(0, \sigma_{P}\right)\\ x_{\text {과목 차}}[c] &=b_{\text{과목 차}}[c] \\ b_{\text{과목 차}}[n] & \sim \operatorname{Normal}\left(0, \sigma_{C}\right) \\x_{\text{수업 }}[i] &=b_{4} \text { Weather }[i] \end{aligned}

In [25]:
#data
df1 = pd.read_csv('data/data-attendance-4-1.txt')
df2 = pd.read_csv('data/data-attendance-4-2.txt')

In [26]:
df1

,PersonID,A,Score
0,1,0,69
1,2,1,145
2,3,0,125
3,4,1,86
4,5,1,158
5,6,0,133
6,7,0,111
7,8,1,147
8,9,0,146
9,10,0,145


In [27]:
df2

,PersonID,CourseID,Weather,Y
0,1,3,B,1
1,1,9,A,1
2,1,1,C,1
3,1,9,A,1
4,1,7,B,1
5,1,9,B,1
6,1,3,C,0
7,1,1,B,1
8,1,9,A,1
9,1,7,A,1


In [70]:
#data
df1 = pd.read_csv('data/data-attendance-4-1.txt')
df2 = pd.read_csv('data/data-attendance-4-2.txt')

#model
model_8_8 ='''
data {
  int N;
  int C;
  int I;
  int<lower=0, upper=1> A[N];
  real<lower=0, upper=1> Score[N];
  int<lower=1, upper=N> PID[I];
  int<lower=1, upper=C> CID[I];
  real<lower=0, upper=1> W[I];
  int<lower=0, upper=1> Y[I];
}

parameters {
  real b[4];
  real b_P[N];
  real b_C[C];
  real<lower=0> s_P;
  real<lower=0> s_C;
}

transformed parameters {
  real x_P[N];
  real x_C[C];
  real x_J[I];
  real x[I];
  real q[I];
  for (n in 1:N)
    x_P[n] = b[2]*A[n] + b[3]*Score[n] + b_P[n];
  for (c in 1:C)
    x_C[c] = b_C[c];
  for (i in 1:I) {
    x_J[i] = b[4]*W[i];
    x[i] = b[1] + x_P[PID[i]] + x_C[CID[i]] + x_J[i];
    q[i] = inv_logit(x[i]);
  }
}

model {
  for (n in 1:N)
    b_P[n] ~ normal(0, s_P);
  for (c in 1:C)
    b_C[c] ~ normal(0, s_C);
  for (i in 1:I)
    Y[i] ~ bernoulli(q[i]);
}

'''
#python interface
label = {'A': 0, 'B': 0.2,'C': 1}

data = {
    'N': df1.shape[0],
    'C': len(np.unique(df2.CourseID)),
    'I': df2.shape[0],
    'A': df1.A,
    'Score' : df1.Score / 200,
    'PID': df2.PersonID,
    'CID' : df2.CourseID,
    'W' : df2.Weather.apply(lambda x: label[x]),
    'Y' : df2.Y,
}

#result
model_8_8 = get_stan_model(model_8_8)
params = model_8_8.sampling(data = data)

# 학생, 수업 별 차이의 사전분포, 출석률이 베르누이를 따른다는 가능도, 즉 수식 중 ~로 표시된 부분을 model
# 나머지 변수 변환은 transformed로 
# score는 200으로 나누어 scaling
# PID는 인스턴스별로 이용되므로, df1.PersonID 아닌 df2.PersonID를 이용

To run all diagnostics call pystan.check_hmc_diagnostics(fit)
